# [실습] 멀티 에이전트 구조

LangGraph의 Agent는 기본적으로 여러 개의 툴을 연결할 수 있습니다.   
그렇지만, 전체 구조를 하나의 에이전트로 구현하기보다는, 문제를 작게 나누는 Divide and Conquer 방식을 활용하여   
개별 문제를 해결하는 에이전트의 연결을 통해 답변하게 하는 것이 효과적일 수 있습니다.   

이는 Agent의 그래프를 따로 구성하고, Human-in-the-loop과 유사하게 `Command()`를 통해 전달하는 방식으로 이루어집니다.   

In [ ]:
!pip install --upgrade langgraph langchain langchain_google_genai google_generativeai tavily-python langchain_community langchain_experimental -q

이번 실습은 한국어 시각화가 필요할 수 있으므로, 코랩에서는 아래 옵션을 실행합니다.

In [ ]:
# Colab/Linux 환경에서 아래 옵션을 실행합니다.
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

위 코드를 실행한 경우, 런타임 --> 세션 다시 시작을 수행해 주세요.

In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', family='NanumBarunGothic')

In [ ]:
import os
os.environ['GOOGLE_API_KEY'] = ''
os.environ["TAVILY_API_KEY"] = ""

from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_google_genai import ChatGoogleGenerativeAI

# Gemini API는 분당 10개 요청으로 제한
# 즉, 초당 약 0.167개 요청 (10/60)
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.167,  # 분당 10개 요청
    check_every_n_seconds=0.1,  # 100ms마다 체크
    max_bucket_size=10,  # 최대 버스트 크기
)

# rate limiter를 LLM에 적용
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    rate_limiter=rate_limiter
)

랭스미스 연동을 설정합니다.

In [ ]:
os.environ['LANGCHAIN_API_KEY'] = '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'
os.environ['LANGCHAIN_PROJECT'] = 'LangGraph_FastCampus'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_TRACING_V2']='true'

## 1. Two Agents 협업 구조    


Tavily Tool과 Python Repl Tool을 이용합니다.

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL

tavily_tool = TavilySearchResults(max_results=5)
repl = PythonREPL()

In [ ]:
from typing import Annotated


# 툴 정보를 보다 자세하게 작성하여 성능을 높입니다.
# LangGraph의 기본 표현을 사용했습니다!
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    result_str = f"Successfully executed:\n```python\n{code}\n```\nStdout: {result}"
    return (
        result_str + "\n\nIf you have completed all tasks, respond with MISSION COMPLETED."
    )

각각의 툴을 연결한 Agent를 만들겠습니다.   
System Prompt을 커스터마이징한 ReAct Agent를 구성합니다.   

최종 출력 조건을 프롬프트에 언급하여 커뮤니케이션의 중단을 표시합니다.


In [ ]:
from langgraph.prebuilt import create_react_agent

def make_system_prompt(suffix: str) -> str:
    return f"""
당신은 여러 AI 어시스턴트와 협업하는 팀원입니다.
목표: 주어진 도구들을 반복적으로 활용해 사용자의 질문에 답하기
당신의 작업물은 다른 에이전트에게 전달되어 최종 답변으로 사용될 것입니다.

[작업 지침]
1. 완벽한 답변이 어렵다면 가능한 부분까지 진행하세요
2. 나머지는 다른 도구를 가진 팀원이 이어서 작업할 것입니다
3. 다른 팀원에게 넘겨주어야 하는 경우, GO! 를 마지막에 출력하세요.
4. 당신과 다른 팀원의 모든 작업이 완료된 경우 MISSION COMPLETED를 마지막에 출력하세요.

팀원이 문제를 잘 해결할 수 있도록, 중요한 부분만 명료하게 설명하세요.

[추가 지침]
{suffix}
"""

# Research Agent 프롬프트
research_agent = create_react_agent(llm, tools=[tavily_tool],
    prompt=make_system_prompt(
        """당신의 이름은 Research 전문가 'Amy' 입니다.
[작업] 정보 검색 및 정리를 수행하세요.
[협업] Chart 생성 전문가 'Brad'와 함께 작업 중입니다."""
    ),
)

# Chart Agent 프롬프트
chart_agent = create_react_agent(llm, [python_repl_tool],
    prompt=make_system_prompt(
        """당신의 이름은 Chart 생성 전문가 'Brad' 입니다.
[작업] 파이썬 코드 실행 및 차트 생성을 수행하세요.
[협업] Research 전문가 'Amy' 와 함께 작업 중입니다."""
    ),
)

State를 구성합니다.

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages : Annotated[list, add_messages]

그래프를 만들고 구성합니다.  

`goto`를 통해, 종료할지/다음 에이전트로 Context를 전달할지를 결정합니다.   

`command`를 통해 다음 경로를 명시적으로 지정할 수 있습니다.   
(이는 Typing에 표시하면, 시각화 과정에 나타납니다.)

In [ ]:
from typing import Literal
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


# Next Node를 선택하는 과정: 노드로 구성하여 그래프에 표시
def get_next_node(last_message, goto: str):
    if "MISSION COMPLETED" in last_message.content:
        return END
    return goto



def research_node(state: State) -> Command[Literal["chart_generator", END]]:

    result = research_agent.invoke(state)
    goto = get_next_node(result["messages"][-1], "chart_generator")


    result["messages"][-1] = HumanMessage(result["messages"][-1].content, name="researcher")
    # name은 실제로 역할이 없으나, 디버깅 및 확인을 위해 구성

    return Command(update={"messages": result["messages"]}, goto=goto)


def chart_node(state: State) -> Command[Literal["researcher", END]]:
    result = chart_agent.invoke(state)
    goto = get_next_node(result["messages"][-1], "researcher")

    result["messages"][-1] = HumanMessage(result["messages"][-1].content, name="chart_generator")
    # name은 실제로 역할이 없으나, 디버깅 및 확인을 위해 구성

    return Command(update={"messages": result["messages"]}, goto=goto)

상호작용 + ReAct Agent의 조합으로 전체 그래프를 구성합니다.

In [ ]:
from langgraph.graph import StateGraph, START

builder = StateGraph(State)
builder.add_node("researcher", research_node)
builder.add_node("chart_generator", chart_node)

builder.add_edge(START, "researcher")
graph = builder.compile()

graph

In [ ]:
events = graph.stream(
    {
        "messages": [
            ("user","2024년 프로야구 순위 그래프로 보여줘")
        ],
    },
    {"recursion_limit": 150},
    stream_mode='updates'
)
for s in events:
    for role in s:
        print(f'Role: {role}')
        print('')
        for message in s[role]['messages']:
            print(message.type,':', message.content)
        print('--------')
    #print(s)
    print("-------------")

## 2. Supervisor

두 에이전트가 서로 소통하는 방식은 고정된 커뮤니케이션 경로로 구성됩니다.

하지만, 차트 에이전트의 코드만을 수정하고 싶거나, 리서치 에이전트의 결과를 바로 수정하고 싶은 경우는 해당 경로가 효과적이지 않은데요, Supervisor 구조를 통해 이를 해결해 보겠습니다.

경로 설정을 위해, agent 목록과 route 목록을 구성합니다.

In [ ]:
agents = ["researcher", "chart_generator"]
routes = agents + ["FINISH"]

from pydantic import BaseModel, Field

class Router(BaseModel):
    reason : str = Field(description='dest를 선택한 이유를 30자 이내로 설명')
    dest: Literal[*routes] = Field(description="다음 작업을 수행할 에이전트를 결정합니다. 만약 모든 작업이 완료되었다면 FINISH를 출력합니다.")


프롬프트를 구성하고, 현재 상황을 판단해 적절한 에이전트를 선택하고 전달하도록 구성합니다.

In [ ]:
system_prompt = f"""
당신은 작업 관리자입니다.
당신의 역할은 주어진 질의를 풀기 위해,

여러 에이전트 간의 대화를 관리하고 조율하는 것입니다.
관리해야 하는 에이전트들은 다음과 같습니다: {agents}

[작업 지침]
1. 사용자 요청에 대한 현재 진행 상황에 따라 다음 작업할 에이전트를 선택하세요
2. 각 에이전트는 자신의 작업 결과와 상태를 보고할 것입니다
3. chart_generator 에이전트의 경우, 코드를 실행하여 전달합니다.코드가 잘 구성되었으면 작업 완료로 간주하세요.

3. 모든 에이전트가 작업을 완료하여 문제가 해결된 경우, FINISH'로 응답하세요. 끝나기 전까지는 계속 다른 에이전트에게 피드백을 제공하세요.

[Notes]
모든 에이전트가 작업에 참여해야만 하는 것은 아닙니다.
작업이 완료된 경우, FINISH로 응답하세요.
"""

In [ ]:
from typing_extensions import TypedDict
class State(TypedDict):
    dest : str
    messages : Annotated[list, add_messages]


In [ ]:
def supervisor(state: State) -> Command[Literal[*agents, "__end__"]]:
    messages = [
        {"role": "system", "content": system_prompt},
    ] + state["messages"]
    response = llm.with_structured_output(Router).invoke(messages)

    print('##SUPERVISOR: ', response.dest, '-->', response.reason)

    goto = response.dest
    if goto == "FINISH":
        goto = END

    return Command(goto=goto, update={"dest": goto})

In [ ]:
def make_system_prompt(suffix: str) -> str:
    return f"""
당신은 툴 사용 에이전트입니다.
아래의 정보를 바탕으로, 문제를 해결하기 위한 과정을 수행하세요.
당신의 영역 밖의 작업은, 다른 에이전트가 수행할 것입니다.

{suffix}
"""

# Research Agent 프롬프트
research_agent = create_react_agent(llm, tools=[tavily_tool],
    prompt=make_system_prompt(
        """당신의 이름은 Research 전문가 'Amy' 입니다.
[작업] 정보 검색 및 정리를 수행하세요.
[협업] Chart 생성 전문가 'Brad'와 함께 작업 중입니다."""
    ),
)

# Chart Agent 프롬프트
chart_agent = create_react_agent(llm, [python_repl_tool],
    prompt=make_system_prompt(
        """당신의 이름은 Chart 생성 전문가 'Brad' 입니다.
[작업] 파이썬 코드 실행 및 차트 생성을 수행하세요.
[협업] Research 전문가 'Amy' 와 함께 작업 중입니다."""
    ),
)


def researcher(state: State) -> Command[Literal["supervisor"]]:

    result = research_agent.invoke(state)

    result["messages"][-1] = HumanMessage(result["messages"][-1].content, name="researcher")
    # name은 실제로 역할이 없으나, 디버깅 및 확인을 위해 구성

    return Command(update={"messages": result["messages"]}, goto='supervisor')


def chart_generator(state: State) -> Command[Literal["supervisor"]]:
    result = chart_agent.invoke(state)

    result["messages"][-1] = HumanMessage(result["messages"][-1].content, name="chart_generator")
    # name은 실제로 역할이 없으나, 디버깅 및 확인을 위해 구성

    return Command(update={"messages": result["messages"]}, goto='supervisor')

In [ ]:
builder = StateGraph(State)

builder.add_node("supervisor", supervisor)
builder.add_node("researcher", researcher)
builder.add_node("chart_generator", chart_generator)

builder.add_edge(START, "supervisor")

In [ ]:
graph = builder.compile()
graph

In [ ]:
events = graph.stream(
    {
        'next':'',
        "messages": [
            ("user","2024년 프로야구 최종 순위 그래프로 보여줘")
        ],
    },subgraphs=True,stream_mode='updates')



for s in events:
    for role in s:
        print(f'Role: {role}')
        print('')
        try:
            for message in s[role]['messages']:
                print(message.type,':', message.content)
        except:
            continue

        print('DEST:', s[role]['dest'])
        print('--------')
    #print(s)
    print("-------------")

In [ ]:
events = graph.stream(
    {
        'next':'',
        "messages": [
            ("user","잠실역 맛집 3개 추천해줘.")
        ],
    },subgraphs=True,stream_mode='updates')



for s in events:
    for role in s:
        print(f'Role: {role}')
        print('')
        try:
            for message in s[role]['messages']:
                print(message.type,':', message.content)
        except:
            continue
        print('DEST:', s[role]['dest'])
        print('--------')
    #print(s)
    print("-------------")

In [ ]:
events = graph.stream(
    {
        'next':'',
        "messages": [
            ("user","2024년 한국 프로야구 10팀의 최종 순위 알려줘.")
        ],
    },subgraphs=True,stream_mode='updates')



for s in events:
    for role in s:
        print(f'Role: {role}')
        print('')
        try:
            for message in s[role]['messages']:
                print(message.type,':', message.content)
        except:
            continue
        print('DEST:', s[role]['dest'])
        print('--------')
    #print(s)
    print("-------------")

이와 같이 간단한 문제에 대해서는, Supervisor가 단순히 노드를 지정하는 것으로 해결할 수 있었습니다.   
만약 복잡한 Reasoning이 필요한 문제라면, Supervisor가 대화에 참여하는 것도 가능합니다.   

Langsmith (https://smith.langchain.com )에서 실행 결과를 확인할 수 있습니다.